# **任务一 线性回归 Linear Regression**

我不准备记录底层的数学逻辑部分的学习，这个项目仅仅只是为了展示我在代码上的理解。

---

## 1. **torch线性层（全连接层）**

torch库最简单的线性映射层，最直观的表现是它能将不同形状的数据以某种规律扩展或压缩维度。

如果不考虑数学因素，对于这么简单的结构其实没有太多需要记录的。

### **使用方法**

它来自于 `torch` 的 `nn` 子类中，通常我们取nn作为别名方便调用 `nn.Linear(input_dim, output_dim)` 方法返回我们需要的线性层对象。

将张量直接作为参数，传入给线性层对象，把它当作一个函数使用即可实现前向传播过程，如以下代码展示

#### 1.1 **特征值张量**

形状为 **[batch_size, dim]** 的输入张量，batch维度是为了方便批量计算额外添加的维度，对于计算是必要的，哪怕每个批次只有一个数据 **[1, dim]**。

In [406]:
import torch
import torch.nn as nn

batch_size = 8
input_tensor = torch.rand((batch_size, 3))

# 定义线性层 nn.Linear
Linear_layer = nn.Linear(3, 5)

# 前向传播过程
output_tensor = Linear_layer(input_tensor)

print('input_tensor:', input_tensor.shape)
print('output_tensor:', output_tensor.shape)

input_tensor: torch.Size([8, 3])
output_tensor: torch.Size([8, 5])


我们注意到对于形状为 **[batch, dim]** 的数据，它会以dim作为被映射的维度进行特征长度的变化。

接下来尝试使用其他常见的张量形状

#### 1.2 **图像张量**

形状为 **[batch_size, channel, height, width]** 的输入张量，channel是通道维度，也就是图像不同颜色的通道，对于不同模式的图像有不同情况。

| 图像模式 | 图像类型       | 通道维度长度 |
|------|------------|--------|
| L    | 灰度图像       | 1      |
| RGB  | 彩色图像       | 3      |
| RGBA | 带透明通道的彩色图像 | 4      |

实际上会有更多模式，我常用的是这三种，实际上RGBA也不太常用，不过不重要，对应场景等理解之后可以随机应变。


In [407]:
batch_size = 8
input_tensor = torch.rand((batch_size, 3, 8, 8))

# 定义线性层 nn.Linear
Linear_layer = nn.Linear(8, 16)

# 前向传播过程
output_tensor = Linear_layer(input_tensor)

print('input_tensor:', input_tensor.shape)
print('output_tensor:', output_tensor.shape)

input_tensor: torch.Size([8, 3, 8, 8])
output_tensor: torch.Size([8, 3, 8, 16])


对于比较常见的图像类型数据 **[batch, channel, height, width]** 它只能从width维度进行变化

看起来似乎拉伸的图像的宽度，但实际上线性数值计算这个过程并没有考虑空间特征，所以不用担心这种变换会影响对图像的处理，因为它从一开始就不该这么用。

在图像中如何使用线性层也是很常见的，但至少不该像这样，后面记录图像任务的时候会再解释。

#### 1.3 **序列张量**

In [408]:
batch_size = 8
input_tensor = torch.rand((batch_size, 5, 8))

# 定义线性层 nn.Linear
Linear_layer = nn.Linear(8, 16)

# 前向传播过程
output_tensor = Linear_layer(input_tensor)

print('input_tensor:', input_tensor.shape)
print('output_tensor:', output_tensor.shape)

input_tensor: torch.Size([8, 5, 8])
output_tensor: torch.Size([8, 5, 16])


对于序列数据 **[batch, seq, dim]** 它的变化看起来比图像合理的多，将特征维度进行了映射，同时没有改变其序列长度。

---

## 2. **模型定义**

对于一个完整的模型通常封装起来会更方便使用和定义，官方方法是使用class类用法实现，以下为定义示例。

In [409]:
class Model(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.Linear_layer = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.Linear_layer(x)

在这里不再强调类用法如何使用，仅对模型定义做出一些说明。

将模型继承自 `torch.nn.Module` 类中，由于 `nn`已经取了别名，所以这里直接使用 `nn.Module`

将需要的层定义在初始化方法中，将前向传播过程定义在forward方法中，forward方法名不能改变，这是官方文档规定的。

---

## 3. **模型调用**

同样是将实例化之后的模型对象直接当作函数前向传播，这与 `torch` 的任何一个层的用法都一样，本质上每个层都可以理解为一个继承自 `nn.Module` 的子类，我们只不过自定义了一个

In [410]:
batch_size = 8
input_tensor = torch.rand((batch_size, 3))

# 实例化模型对象
model = Model(3, 5)

# 前向传播过程
output_tensor = model(input_tensor)

print('model:\n', model)
print('input_tensor:', input_tensor.shape)
print('output_tensor:', output_tensor.shape)

model:
 Model(
  (Linear_layer): Linear(in_features=3, out_features=5, bias=True)
)
input_tensor: torch.Size([8, 3])
output_tensor: torch.Size([8, 5])


---

## 4. **线性回归的训练**

介绍完第一个需要认识的层之后可以开始关于它的第一个任务——线性回归

先前学习过线性回归，这个任务是深度学习的基础，所以从它来引入深度学习框架的使用再合适不过

### 4.1 **数据生成**

本次任务我们需要拟合的是一条直线，实际上只需要两组“特征值”数据，分别是 $y = wx + b$ 的 $x$ 和 $y$

其中 $w$ 和 $b$ 就是模型需要训练得到的模型参数。

In [411]:
w, b, noise_range = 2.3, -8, 0.1
batch_size = 100
# 输入 x
x = torch.randint(low=-50, high=51, size=(batch_size, 1), dtype=torch.float32)

# 输出 y
y = w * x + b
# 噪声
noise = 2 * (torch.rand_like(y) * noise_range) - 1

# 4. 整数叠加噪声，得到最终结果
y = y + noise

x.shape, y.shape

(torch.Size([100, 1]), torch.Size([100, 1]))

这里我们生成了200个随机数据x，然后使用 $y = kx + b$ 计算出 $y$ 作为输出

其中 $x$ 作为 input 也就是输入，$y$ 作为 label 也就是标注数据。这里的强调主要是方便理解常见的深度学习命名方法。

### 4.2 **定义模型、损失函数和优化器**

学习机器学习时了解到，线性回归任务通常使用均方误差函数，torch框架中正好有封装这种损失函数 `nn.MSELoss()`。

优化器的类别很多，这里用比较常用的，具体有哪些优化器它们有什么区别这里不强调，因为只是为了展示不同任务在模型结构和层用法。

lr 就是我们线性回归时用到的学习率。

In [412]:
# 线性回归的输入和输出均只有一个维度
model = Model(1, 1)
# 定义损失函数，这里使用均方误差损失（MSELoss）
criterion = nn.MSELoss()
# 定义优化器，使用随机梯度下降（SGD）来更新权重和偏置
optimizer = torch.optim.SGD(model.parameters(), lr=0.0008)

### 4.3 **迭代训练**

流程与线性回归几乎无异。需要注意的是对于 `torch` 框架很多功能都是自动的，并且需要按照其定义的代码段逐个执行。

- 前向传播: `output = model(x)`
- 计算损失: `loss = criterion(output, y)`
- 梯度清零: `optimizer.zero_grad()`
- 反向传播: `loss.backward()`
- 更新权重和偏置: `optimizer.step()`

最后根据具体情况增加必要的指标，方便实时监测训练情况。

In [413]:
epochs = 1000
for epoch in range(1000):
    # 前向传播，得到预测值
    output = model(x)
    # 计算损失
    loss = criterion(output, y)
    # 梯度清零，因为在每次反向传播前都要清除之前累积的梯度
    optimizer.zero_grad()
    # 反向传播，计算梯度
    loss.backward()
    # 更新权重和偏置
    optimizer.step()

    if (epoch + 1) % 50 == 0:
        print(f'[epoch {epoch+1}]loss:', loss.item())


[epoch 50]loss: 77.56211853027344
[epoch 100]loss: 66.16356658935547
[epoch 150]loss: 56.440242767333984
[epoch 200]loss: 48.14592361450195
[epoch 250]loss: 41.070560455322266
[epoch 300]loss: 35.035037994384766
[epoch 350]loss: 29.886518478393555
[epoch 400]loss: 25.494667053222656
[epoch 450]loss: 21.74828338623047
[epoch 500]loss: 18.5524845123291
[epoch 550]loss: 15.826355934143066
[epoch 600]loss: 13.500873565673828
[epoch 650]loss: 11.517169952392578
[epoch 700]loss: 9.825000762939453
[epoch 750]loss: 8.381522178649902
[epoch 800]loss: 7.150174140930176
[epoch 850]loss: 6.099789619445801
[epoch 900]loss: 5.203781604766846
[epoch 950]loss: 4.439455986022949
[epoch 1000]loss: 3.787461042404175


模型损失值逐步降低，至少说明模型在训练集中正在进行有效的学习。

为什么强调训练集？涉及到有关**过拟合**的情况，这里同样不做过多记录。

### 4.4 **测试模型**

误差在可接受的范围内

In [414]:
model.eval()
x1 = torch.randint(low=-100, high=101, size=(1, 1), dtype=torch.float32)
x2 = torch.randint(low=-100, high=101, size=(1, 1), dtype=torch.float32)

y_p1 = model(x1)
y_p2 = model(x2)
y1 = w * x1 + b
y2 = w * x2 + b

print('真实值1:', y1.item())
print('预测值1:', y_p1.item())

print('真实值2:', y2.item())
print('预测值2:', y_p2.item())


真实值1: -217.3000030517578
预测值1: -216.7677764892578
真实值2: 148.39999389648438
预测值2: 149.8332977294922


### 4.5 观察模型参数

这里获取模型的权重 $w$ 和偏置 $b$，发现模型虽然仍存在些许误差，但总体在往目标直线拟合。

In [415]:
weight = model.Linear_layer.weight.data  # 获取权重
bias = model.Linear_layer.bias.data      # 获取偏置
print(f"训练权重(w): {weight.item()}, 训练偏置(b): {bias.item()}")
print(f"真实权重(w): {w}, 真实偏置(b): {b}")

训练权重(w): 2.3056671619415283, 训练偏置(b): -6.952065944671631
真实权重(w): 2.3, 真实偏置(b): -8


## 5. **总结**

这是第一个关于 `torch` 框架的任务，足够简单也足够好理解，很好的将机器学习的线性回归引入的深度学习框架的使用。